In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
data=pd.read_csv('apache-spark-docker-main\docker\workspace\AB_NYC_2019.csv')
mean_control=data['price'].mean()
var_control=np.var(data['price'])
print(mean_control,var_control)

152.7206871868289 57672.84569843359


In [12]:
def comFunc(res, x):
    return res[0] + x, res[1] + 1

def integralFunc(temp):
    # Сумма значений и количествои количество для расчета дисперсии
    com_value, com_cnt,var_dev = reduce(lambda res, item: (res[0] + item[0], res[1] + item[1],res[2]+(item[1]-1)), temp,(0,0,0))
    com_mean = com_value / com_cnt
    
    # Теперь считаем общую дисперсию из дисперсий кусков
    com_temp = reduce(lambda res, item: res + (item[2] * (item[1] - 1) + item[1] * ((item[0] - com_mean) ** 2)), temp, 0)
    com_var = com_temp / (com_cnt - 1) if com_cnt > 1 else 0  # Избегаем деления на ноль

    print(np.mean(temp[2]))#посчитаем просто среднее по дисперсиям - для того чтоб подтвердить, что посчитанная по кускам оющая дисперсия близка к этому значению
    
    return com_mean, com_var/var_dev #почти равно com_var/com_cnt


chunksize = 200
temp = []
for chunk in pd.read_csv('apache-spark-docker-main/docker/workspace/AB_NYC_2019.csv', chunksize=chunksize):
    # Считаем сумму, количество и дисперсию для каждого чанка
    sum_prices, count_prices = reduce(comFunc, chunk['price'], (0, 0))
    var_price = np.var(chunk['price'], ddof=1)  # Используем ddof=1 для выборочной дисперсии
    temp.append((sum_prices, count_prices, var_price))
    

print(integralFunc(temp))


19384.865360134005
(152.7206871868289, 19773.035129751934)
